![Example of HMM Model -- Fever](../images/example_of_hmm_fever.png)

refer to [wikipedia](https://en.wikipedia.org/wiki/Viterbi_algorithm#Example) for details

In [1]:
import numpy as np

In [2]:
# 状态集合 Q
states = ('Healthy', 'Fever')
# 观测集合 V
observations = ('normal', 'cold', 'dizzy')
# 初始状态概率向量 π
start_probability = {'Healthy': 0.6, 'Fever': 0.4}
# 状态转移矩阵 A
transition_probability = {
    'Healthy': {'Healthy': 0.7, 'Fever': 0.3},
    'Fever': {'Healthy': 0.4, 'Fever': 0.6},
}
# 观测概率矩阵 B
emission_probability = {
    'Healthy': {'normal': 0.5, 'cold': 0.4, 'dizzy': 0.1},
    'Fever': {'normal': 0.1, 'cold': 0.3, 'dizzy': 0.6},
}

In [3]:
def convert_map_to_vector(probs_map, col_label2id):
    """conver to 1d vector"""
    v = np.zeros(len(col_label2id), dtype=float)
    for col, value in probs_map.items():
        v[col_label2id[col]] = value
    return v


def convert_map_to_matrix(probs_map, row_label2id, col_label2id):
    """convert to matrix"""
    m = np.zeros((len(row_label2id), len(col_label2id)), dtype=float)
    for row, cols in probs_map.items():
        for col, value in cols.items():
            m[row_label2id[row]][col_label2id[col]] = value
    return m

In [4]:
def generate_index_map(lables):
    id2label = {}
    label2id = {}
    for idx, label in enumerate(lables):
        id2label[idx] = label
        label2id[label] = idx
    return id2label, label2id
 
states_id2label, states_label2id = generate_index_map(states)
observations_id2label, observations_label2id = generate_index_map(observations)
print(states_id2label, states_label2id)
print(observations_id2label, observations_label2id)

{0: 'Healthy', 1: 'Fever'} {'Healthy': 0, 'Fever': 1}
{0: 'normal', 1: 'cold', 2: 'dizzy'} {'normal': 0, 'cold': 1, 'dizzy': 2}


In [5]:
A = convert_map_to_matrix(transition_probability, states_label2id, states_label2id)
print('状态转移矩阵 A: ', A)
B = convert_map_to_matrix(emission_probability, states_label2id, observations_label2id)
print('观测概率矩阵 B: ', B)
observations_index = [observations_label2id[o] for o in observations]
pi = convert_map_to_vector(start_probability, states_label2id)
print('初始状态概率向量 π: ', pi)

状态转移矩阵 A:  [[0.7 0.3]
 [0.4 0.6]]
观测概率矩阵 B:  [[0.5 0.4 0.1]
 [0.1 0.3 0.6]]
初始状态概率向量 π:  [0.6 0.4]


In [6]:
# 随机生成观测序列和状态序列    
def simulate(T):

    def draw_from(probs):
        """
        1.np.random.multinomial:
        按照多项式分布，生成数据
        >>> np.random.multinomial(20, [1/6.]*6, size=2)
                array([[3, 4, 3, 3, 4, 3],
                       [2, 4, 3, 4, 0, 7]])
         For the first run, we threw 3 times 1, 4 times 2, etc.  
         For the second, we threw 2 times 1, 4 times 2, etc.
        2.np.where:
        >>> x = np.arange(9.).reshape(3, 3)
        >>> np.where( x > 5 )
        (array([2, 2, 2]), array([0, 1, 2]))
        """
        return np.where(np.random.multinomial(1, probs) == 1)[0][0]

    observations = np.zeros(T, dtype=int)
    states = np.zeros(T, dtype=int)
    states[0] = draw_from(pi)  # pi is initial state prob distribution
    observations[0] = draw_from(B[states[0],:])
    for t in range(1, T):
        states[t] = draw_from(A[states[t-1],:])
        observations[t] = draw_from(B[states[t],:])
    return observations, states

In [7]:
observations_data, states_data = simulate(10)
print("病人的状态: ", [states_id2label[index] for index in states_data])
print("病人的观测: ", [observations_id2label[index] for index in observations_data])

病人的状态:  ['Fever', 'Fever', 'Fever', 'Fever', 'Fever', 'Fever', 'Fever', 'Healthy', 'Fever', 'Healthy']
病人的观测:  ['dizzy', 'cold', 'dizzy', 'dizzy', 'dizzy', 'normal', 'dizzy', 'normal', 'cold', 'dizzy']


In [8]:
simulate(10)

(array([1, 0, 2, 1, 1, 1, 2, 2, 2, 2]), array([0, 0, 1, 1, 1, 1, 1, 0, 1, 1]))